# Action on Tumblr
## Web-scraper
-------------
Description: We all need to work together to determine the future of the world. It’s time to #takeaction.

### Import libraries and data
---

In [1]:
#!pip install requests[security]
from bs4 import BeautifulSoup
import urllib

import csv
import requests
import pandas as pd
import numpy as np

r = urllib.urlopen('ActiononTumblr.htm').read()
soup = BeautifulSoup(r, "html5lib")
print type(soup)

<class 'bs4.BeautifulSoup'>


### Load Website Content and Generate Dataset
---

In [4]:
# Write links from Action Tumblr into .csv
path = ''
f = csv.writer(open(path + "tumblr.csv", "w"))
f.writerow(["Link", "Number of Notes", "Date"]) # Write column headers as the first line

post = soup.find_all("div", class_="date-note-wrapper")

for each in post:
    link = list(each)[1].find('a').get('href')
    number = list(each)[1].find('a').get_text()
    date = list(each)[3].get_text()
    
    f.writerow([link, number, date])
    
df = pd.read_csv(path + "tumblr.csv")
df.head()

,Link,Number of Notes,Date
0,https://action.tumblr.com/post/159839983820/pb...,"1,386 notes","Apr 21st, 2017"
1,https://action.tumblr.com/post/159839945455/wh...,"1,653 notes","Apr 21st, 2017"
2,https://action.tumblr.com/post/159839943050/a-...,323 notes,"Apr 21st, 2017"
3,https://action.tumblr.com/post/159839941675/ho...,320 notes,"Apr 21st, 2017"
4,https://action.tumblr.com/post/159839938215/to...,79 notes,"Apr 21st, 2017"


### Using Requests, BeautifulSoup and Pytesseract to scrap the Web:
---
** Steps: **
- Import libraries
- Define new columns for our dataset
- Iterate through all rows
- Get asker name and question
- Get metatags from the question
- Iterate through the replies
- The name of the person who answers is encoded in an image. Use pytesseract to transform the image into text and save that name
- Remove any unwanted character
- Get replies related to each person and save their answers

In [35]:
import io
import pytesseract
from PIL import Image

df['Asker'] = ''
df['Asker_question'] = ''
df['Tags'] = ''
df['EXP_1_who'] = ''
df['EXP_1_answer'] = ''
df['EXP_2_who'] = ''
df['EXP_2_answer'] = ''
df['EXP_3_who'] = ''
df['EXP_3_answer'] = ''
df['EXP_4_who'] = ''
df['EXP_4_answer'] = ''
df['EXP_5_who'] = ''
df['EXP_5_answer'] = ''

for index, row in df.iterrows():
    
    url = row['Link']
    page = requests.get(url)
    page_data = BeautifulSoup(page.content, 'html.parser')
    
    row['Asker'] = page_data.find_all("p", class_="asker")[0].find('strong').get_text()
    row['Asker_question'] = page_data.find_all("div", class_="asker-question")[0].get_text().strip()
    
    mylist = []
    tags_all = page_data.find_all("section", class_ = "inline-meta post-extra has-tags")[0].find_all('a')
    for each in tags_all:
        mylist.append(each.get_text())
    row['Tags'] = ','.join(mylist)
    
    try:
        raw = page_data.find('div', class_ = 'answerer-answer')
        if len(raw) > 1:
            replies = raw
        else:
            raw = page_data.find_all("div", class_ = "replies")[0].contents
            replies = raw[1:-1]
    except:
        raw = page_data.find_all("div", class_ = "replies")[0].contents
        replies = raw[1:-1]
    
    i = 0
    mylist = []
    
    for each in replies:
        try:
            answer = each.get_text()
            if len(answer) > 1:
                mylist.append(answer.strip())
                answer_txt = ' '.join(mylist)
                row['EXP_' + str(i) + '_answer'] = answer_txt
            elif (each.img['src']) is not None:
                i += 1
                response = requests.get(each.img['src'])
                img = Image.open(io.BytesIO(response.content))
                img_txt = pytesseract.image_to_string(img)
                text = " ".join(img_txt.split()[2:])
                row['EXP_' + str(i) + '_who'] = text
                mylist = []
        except:
            next

### Using Tesseract-OCR to get name of each expert
---
**Requisites:**
- Python-tesseract requires python 2.5+ or python 3.x
- You will need the Python Imaging Library (PIL) (or the Pillow fork). Under Debian/Ubuntu, this is the package python-imaging or python3-imaging.
- Install Google Tesseract OCR (different steps depending on the version of Linux you are running)
---
** Instructions & Notes:**
- type: **!sudo pip install pytesseract**
- **[pytesseract.py]** Change: import Image => from PIL import Image
- Download tessdata from Github, https://github.com/tesseract-ocr/tessdata/tree/3.04.00 in /usr/local/share/tessdata
---
** Example: **

In [ ]:
# import io
# import requests
# import pytesseract
# from PIL import Image

# #image = 'https://68.media.tumblr.com/af07c593f952c90616cc557bd560db61/tumblr_inline_oo8zobDy5I1u3807l_1280.jpg'
# image = 'https://68.media.tumblr.com/0c33775a5aaeee35017c85452979a77b/tumblr_inline_oll1e4R2Vu1qeblmp_1280.jpg'

# response = requests.get(image)
# img = Image.open(io.BytesIO(response.content))
# text = pytesseract.image_to_string(img)
# print( text )

### Export Final Dataset to .CSV
---

In [38]:
df.to_csv('tumblr_export.csv', encoding='utf-8', index = False)